## Understanding " NEURAL PROBABILISTIC LANGUAGE MODEL "

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:


# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]



['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  
  for ch in w + '.':
    ix = stoi[ch]
   
    X.append(context)
   
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [5]:
X.shape, Y.shape
print(X)


tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])


In [6]:
X[1][2]

tensor(5)

In [7]:
C = torch.randn((27,6))  # we are creating a less dimesion vector for inputing large amount of data
C[X]

tensor([[[-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01],
         [-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01],
         [-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01]],

        [[-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01],
         [-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01],
         [-4.1507e-01,  1.0066e+00, -1.8788e-01,  3.9494e-01,  6.2890e-01,
          -4.9750e-01]],

        [[-3.4699e-01, -3.8890e-01,  1.2909e+00,  6.3136e-01,  9.3143e-01,
          -6.0442e-01],
         [-4.1507e-01,  1.0066e+00, -1.8788e-01,  3.9494e-01,  6.2890e-01,
          -4.9750e-01],
         [-6.5308e-01, -1.8026e+00, -4.6173e-01,  1.9262e-01,  3.6286e-01,
          -5.8848e-01]],

        ...,

        [[-1.0695e+00,  1.7101e+00, -7.1060e-01, -2.8857e-04, -3.9631e-01,
          -2.4

In [26]:
emb = C[X]
emb[:3].shape



torch.Size([3, 3, 6])

In [9]:
W1 = torch.randn((18, 100))
b1 = torch.randn(100)

In [10]:
h = torch.tanh(emb.view(-1, 18) @ W1 + b1)

In [11]:


h.shape




torch.Size([228146, 100])

In [12]:


W2 = torch.randn((100, 27))
b2 = torch.randn(27)



In [13]:


logits = h @ W2 + b2



In [14]:


logits.shape



torch.Size([228146, 27])

In [15]:
counts = logits.exp()


prob = counts / counts.sum(1, keepdims=True)



prob.shape







torch.Size([228146, 27])

In [16]:
loss = -prob[torch.arange(228146), Y].log().mean()
loss

tensor(17.6727)

In [21]:
logits.shape


torch.Size([228146, 27])

In [23]:
'''
This loss is computationally better than the loss that we have calculated above because of 
 --> less creation of tensors internally  
 --> It smoothens the tensors by subracting eith max tensor in that row to avoid the ouliers of large positive values

'''
loss = F.cross_entropy(logits, Y)
loss

tensor(17.6727)